# ElectroMKM class import and object instantiation

In [1]:
import sys
sys.path.insert(0, "../") 
from electromkm import electroMKM

The demo system deal with the Hydrogen Evolution Reaction (HER) with random values.

In [2]:
model = electroMKM('CO2R_K',
                   'rm.mkm', 
                   'g.mkm', 
                    t_ref=298)

In [3]:
for i in list(model.gpcet_dict.values()):
    print(i)

2.0
2.0


# Model exploration


To investigate the characteristics of the system under study, several attributes can be easily inspected to check general information like number of elementary reactions, energetics, reaction network, etc.

In [4]:
print(model.species_gas)

['C1O2(g)', 'C1O1(g)', 'H2O(g)', 'H2(g)']


### Defined species in the system

N.B. H(e) is used to define H+ + e-.

In [5]:
model.species_tot

['i000000',
 'i010101',
 'i101101',
 'i102101a',
 'i112102',
 'H2O(e)',
 'C1O2(g)',
 'C1O1(g)',
 'H2O(g)',
 'H2(g)']

### Visualize Gibbs energetics of the system

Reaction types: 'ads'=adsorption
                'des'=desorption
                'sur'=surface reaction. 
The suffix "+e" means that that elementary reaction is a charge-transfer step.

In [6]:
model.df_gibbs

,,DGR / eV,DG barrier / eV,DG reverse barrier / eV
R1,ads,0.504,0.504,0.000
R2,ads,0.476,0.476,0.000
R3,des+b_e,-0.069,0.590,0.659
R4,sur+b_e,0.332,0.919,0.587
R5,sur+b_e,0.331,1.610,1.279
R6,ads,0.662,0.662,0.000


In [7]:
model.dh_barrier

array([0.044, 0.02 , 0.551, 0.868, 1.613, 0.326])

### Stoichiometric matrix of the reaction network

In [8]:
model.df_system

,R1,R2,R3,R4,R5,R6
,ads,ads,des+b_e,sur+b_e,sur+b_e,ads
species,,,,,,
i000000,-1,-1,0,0,-1,-2
i010101,0,0,0,0,1,2
i101101,0,1,1,0,0,0
i102101a,1,0,0,-1,0,0
i112102,0,0,-1,1,0,0
H2O(e),0,0,-1,-1,-1,0
C1O2(g),-1,0,0,0,0,0
C1O1(g),0,-1,0,0,0,0


### Stoichiometric vector of the global reactions

In [9]:
model.gr_string

['C1O2(g) + 2H2O(e) + K+(cat) -> C1O1(g) + H2O(g)',
 '2H2O(e) + K+(cat) -> H2(g)']

In [10]:
model.target_label

'C2C'

model.stoich_numbers tells us that the first elementary reaction must be multiplied by two and summed up to the second one in order to get the global reaction. This is useful for checking the thermodynamic consistency of the developed models.

# Microkinetic runs and Tafel plot

Up to now, it is possible to run steady state runs via the electroMKM.kinetic_run() function.
The main inputs that must be provided are the applied overpotential and the pH of the electrolyte solution.
The output of the function is a Python dictionary containing information related to the performed simulation.

In [11]:
model.set_ODE_params(t_final=100)

Final integration time = 100s
Relative tolerance = 1e-12
Absolute tolerance = 1e-64


'Changed ODE solver parameters.'

### Steady state simulation

In [12]:
help(electroMKM.kinetic_run)

Help on function kinetic_run in module electromkm:

kinetic_run(self, overpotential:float, pH:float, cation_conc:float, potential_dl:float, cation_ener_error:float, initial_sur_coverage:list=None, temperature:float=298.0, pressure:float=100000.0, gas_composition:numpy.ndarray=None, verbose:int=0, jac:bool=False)
    Simulates a steady-state electrocatalytic run at the defined operating conditions.        
    Args:
        overpotential(float): applied overpotential [V vs SHE].
        pH(float): pH of the electrolyte solution [-].
        cation_conc(float): cation_conc at OHP [M]
        potential_dl (float): potential difference across double layer [V]
        temperature(float): Temperature of the system [K].
        pressure(float): Absolute pressure of the system [Pa].
        initial_conditions(nparray): Initial surface coverage array[-].
        verbose(int): 0=print all output; 1=print nothing.        
    Returns:
        (dict): Report of the electrocatalytic simulation.



In [18]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.46)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-1.244, 3.34, 3.11,0.62,0.13, gas_composition = array , jac = True)

[0.46 0.   1.   0.  ]
CO2R_K: Microkinetic run
Overpotential = -1.244V vs SHE    pH = 3.34
Overpotential = -1.04694V vs RHE
Temperature = 298.0K    Pressure = 1.0bar

C2C Current density: 2.14e-09 mA cm-2
C2C Selectivity: 99.93%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 0.28 s


In [20]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.0609)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-1.804, 6.546, 5.3803,1.016,-0.13, gas_composition = array , jac = True)


[0.0609 0.     1.     0.    ]
CO2R_K: Microkinetic run
Overpotential = -1.804V vs SHE    pH = 6.546
Overpotential = -1.417786V vs RHE
Temperature = 298.0K    Pressure = 1.0bar

C2C Current density: 2.02e+00 mA cm-2
C2C Selectivity: 99.15%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 3.55 s


In [38]:
model.grl_pcet

{'C2C': [1, 2.0, 0.005474664327364803], 'HER': [5, 2.0, 0.005474664327364803]}

In [39]:
exp['j_C2C']

6.759042241412582

In [22]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.014)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-2.1, 8.3, 3.8134,1.28,-0.13, gas_composition = array , jac = True)

[0.014 0.    1.    0.   ]
CO2R_K: Microkinetic run
Overpotential = -2.1V vs SHE    pH = 8.3
Overpotential = -1.6103V vs RHE
Temperature = 298.0K    Pressure = 1.0bar

C2C Current density: 1.33e+00 mA cm-2
C2C Selectivity: 11.03%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 0.26 s


In [41]:
model.df_gibbs_e

,,DGR_e / eV,DG barrier_e / eV,DG reverse barrier_e / eV
R1,ads,0.504000,0.504,0.000000
R2,ads,0.476000,0.476,0.000000
R3,des+b_e,-1.678779,0.000,1.678779
R4,sur+b_e,-1.277779,0.000,1.277779
R5,sur+b_e,-1.278779,0.560,1.838779
R6,ads,0.662000,0.662,0.000000


In [42]:
import numpy as np
array=[]
for i in model.species_gas:
    if i=='C1O2(g)':
        array.append(0.039)
    elif i=='H2O(g)':
        array.append(1)
    else:
        array.append(0)
array=np.array(array)
print(array)
exp = model.kinetic_run(-1.9, 6.63, 5.5, 1.1, gas_composition = array , jac = True)

[0.039 0.    1.    0.   ]
CO2R_K: Microkinetic run
Overpotential = -1.9V vs SHE    pH = 6.63
Overpotential = -1.50883V vs RHE
Temperature = 298.0K    Pressure = 1.0bar
cation_ads_ener: 0.16, site_adj_factor: 0.011
cation_ads_ener: 0.16, site_adj_factor: 0.011
final adjust_factor is : [0.010708921711386297, 0.010708921711386297]

C2C Current density: 1.24e+00 mA cm-2
C2C Selectivity: 94.43%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 0.15 s


In [43]:
model.df_gibbs_e

,,DGR_e / eV,DG barrier_e / eV,DG reverse barrier_e / eV
R1,ads,0.504000,0.504,0.000000
R2,ads,0.476000,0.476,0.000000
R3,des+b_e,-1.577414,0.000,1.577414
R4,sur+b_e,-1.176414,0.000,1.176414
R5,sur+b_e,-1.177414,0.660,1.837414
R6,ads,0.662000,0.662,0.000000


In [21]:
model.target_label

'C2C'

In [22]:
target_label=model.target_label


In [23]:
cation_conc_array=[3.8348,4.2943,4.6758,4.9791,5.2104,5.3803,5.5004,5.5826,5.6371]
j_array=[]
for conc in cation_conc_array:
    exp = model.kinetic_run(-1.8, 8, conc, 1.8-0.8, gas_composition = array , jac = True)
    j_array.append(exp['j_C2C'])

CO2R_K: Microkinetic run
Overpotential = -1.8V vs SHE    pH = 8
Overpotential = -1.328V vs RHE
Temperature = 298.0K    Pressure = 1.0bar
cation_ads_ener: 0.18, site_adj_factor: 0.003
cation_ads_ener: 0.18, site_adj_factor: 0.003
final adjust_factor is : [0.0034518929511542134, 0.0034518929511542134]

C2C Current density: 5.94e-01 mA cm-2
C2C Selectivity: 99.11%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 0.15 s
CO2R_K: Microkinetic run
Overpotential = -1.8V vs SHE    pH = 8
Overpotential = -1.328V vs RHE
Temperature = 298.0K    Pressure = 1.0bar
cation_ads_ener: 0.18, site_adj_factor: 0.004
cation_ads_ener: 0.18, site_adj_factor: 0.004
final adjust_factor is : [0.0038639134176290282, 0.0038639134176290282]

C2C Current density: 5.94e-01 mA cm-2
C2C Selectivity: 99.11%
Most Abundant Surface Intermediate: i000000 Coverage: 100.00% 
CPU time: 0.13 s
CO2R_K: Microkinetic run
Overpotential = -1.8V vs SHE    pH = 8
Overpotential = -1.328V vs RHE
Temperature = 298

In [23]:
j_array

[2.559432666464913,
 2.866113393679811,
 3.1207351490925563,
 3.3231644492088592,
 3.477539315424827,
 3.5909344286568823,
 3.6710918922814315,
 3.7259540363292825,
 3.762328572188492]

Once steady state conditions have been checked, the solution can be easily analyzed. the main output consists of steady state surface coverage and reaction rate in term of current density.

In [ ]:
exp['theta']

In [ ]:
exp['r']

Negative current density means reduction is occurring, while positive values means that reaction is evolving in the opposite direction. Values of current density are stored in mA cm-2.

In [ ]:
exp['j_C2C']

### Tafel plot

In [17]:
import numpy as np
n = np.arange(-1.11,-2.81,-0.2)
#n = np.array([-1.5,-1.75,-2,-2.25])
print(n)

[-1.11 -1.31 -1.51 -1.71 -1.91 -2.11 -2.31 -2.51 -2.71]


In [18]:
help(electroMKM.tafel_plot)

Help on function tafel_plot in module electromkm:

tafel_plot(self, reaction_label:str, overpotential_vector:numpy.ndarray, pH:float, cation_conc:float, potential_dl:float, initial_sur_coverage:list=None, temperature:float=298.0, pressure:float=100000.0, gas_composition:numpy.ndarray=None, verbose:int=0, jac:bool=True)
    Returns the Tafel plot for the defined potential range.
    Args:
        reaction_label(str): Label of the reaction of interest.
        overpotential_vector(ndarray): applied overpotential vector [V].
        pH(float): pH of the electrolyte solution [-].
        
        initial_conditions(ndarray): initial surface coverage and gas composition [-]
        temperature(float): Temperature of the system [K].
        pressure(float): Absolute pressure of the system [Pa].
        verbose(bool): 0=; 1=.
        jac(bool): Inclusion of the analytical Jacobian for ODE numerical solution.



In [19]:
model.tafel_plot("C2C", n ,7, 1, n-0.8, gas_composition = array, jac=True)

CO2R_K: Tafel slope experiment for C2C
Temperature: 298.0 K    Pressure: 1 bar    pH: 7



ValueError: setting an array element with a sequence.

In [ ]:
rxn_list = ['C2C','HER']
label = ['CO','H2']#,'Methane']

color = ['r','g']

model.j_potential(rxn_list, label, n ,7, color, gas_composition = array, jac=True,set_y_lim=[-10,25])

In [16]:
model.drc_full('C2C',-2.5, 7,1, gas_composition = array,jac='True')

CO2R_K: Full DRC and DSC analysis wrt C2C global reaction
Temperature = 298.0K    Pressure = 1.0bar
Gas composition: C1O2=100.0%  C1O1=0.0%  H2O=100.0%  H2=0.0%  

R1
DRC = 0.50

R2
DRC = 0.00

R3
DRC = -0.00

R4
DRC = -0.00

R5
DRC = -0.00

R6
DRC = 0.00

i000000
DRC = -1.00

i010101
DRC = -0.00

i101101
DRC = -0.00

i102101a


KeyboardInterrupt: 

### Check for the analytical Jacobian matrix

In order to check if the implemented analytical Jacobian is correct or not, we run the same simulation with and without the analytical Jacobian: If the solutions are the same, the Jacobian is correct. If not, it means that the wrong Jacobian drives the system away from the correct solution. 

#### Simulation without analytical Jacobian

In [ ]:
exp1 = model.kinetic_run(-0.1, 7, jac=False)

#### Simulation with analytical Jacobian

In [ ]:
exp2 = model.kinetic_run(-0.1, 7, jac=True)

Observe the difference in the CPU time required to integrate the system without and with the Jacobian!